In [23]:
from pathlib import Path
from PIL import Image

In [24]:
DOWNLOAD_FOLDER = Path("data")

ROOT_FOLDER = Path.cwd()
output_pdf_folder = ROOT_FOLDER / "pdf"
output_pdf_folder.mkdir(exist_ok=True)

In [25]:
def make_pdfs(download_folder: Path):
    for manga_dir in download_folder.iterdir():
        if not manga_dir.is_dir():
            continue

        print(f"\n=== Обробка манги: {manga_dir.name} ===")

        all_chapter_images = []

        # проходимо по всіх розділах
        for chapter_dir in sorted(manga_dir.iterdir(), key=lambda x: x.name):
            if not chapter_dir.is_dir():
                continue

            # всі файли зображень, сортуємо за назвою
            images = sorted(chapter_dir.glob("*.*"), key=lambda x: x.name)
            if not images:
                print(f"  {chapter_dir.name} — немає картинок")
                continue

            # створюємо PDF для розділу
            pdf_path = manga_dir / f"{chapter_dir.name}.pdf"
            img_list = [Image.open(img_file).convert("RGB") for img_file in images]
            img_list[0].save(pdf_path, save_all=True, append_images=img_list[1:])
            print(f"  PDF створено: {pdf_path}")

            # додаємо всі зображення цього розділу для загального PDF
            all_chapter_images.extend(img_list)

        # створюємо загальний PDF на всю мангу
        if all_chapter_images:
            total_pdf_path = manga_dir / f"{manga_dir.name}_full.pdf"
            all_chapter_images[0].save(total_pdf_path, save_all=True, append_images=all_chapter_images[1:])
            print(f"\n=== Загальний PDF для манги створено: {total_pdf_path} ({len(all_chapter_images)} сторінок) ===")

In [26]:
make_pdfs(DOWNLOAD_FOLDER)


=== Обробка манги: Головні герої опинились у пастці мого маєтку ===
  PDF створено: data/Головні герої опинились у пастці мого маєтку/0001_84269-golovni-geroji-opinilis-u-pastci-mogo-mayetku-tom-1-rozdil-0.pdf
  PDF створено: data/Головні герої опинились у пастці мого маєтку/0002_84317-golovni-geroji-opinilis-u-pastci-mogo-mayetku-tom-1-rozdil-1.pdf
  PDF створено: data/Головні герої опинились у пастці мого маєтку/0003_84348-golovni-geroji-opinilis-u-pastci-mogo-mayetku-tom-1-rozdil-2.pdf
  PDF створено: data/Головні герої опинились у пастці мого маєтку/0004_84407-golovni-geroji-opinilis-u-pastci-mogo-mayetku-tom-1-rozdil-3.pdf
  PDF створено: data/Головні герої опинились у пастці мого маєтку/0005_84424-golovni-geroji-opinilis-u-pastci-mogo-mayetku-tom-1-rozdil-4.pdf
  PDF створено: data/Головні герої опинились у пастці мого маєтку/0006_84487-golovni-geroji-opinilis-u-pastci-mogo-mayetku-tom-1-rozdil-5.pdf
  PDF створено: data/Головні герої опинились у пастці мого маєтку/0007_84537-go

In [ ]:
for manga_dir in DOWNLOAD_FOLDER.iterdir():
    if not manga_dir.is_dir():
        continue

    for pdf_file in manga_dir.glob("*_full.pdf"):
        pdf_file.rename(output_pdf_folder / pdf_file.name)

print(f"\n=== Всі загальні PDF файли переміщено до папки: {output_pdf_folder} ===")


=== Всі загальні PDF файли переміщено до папки: /Users/aleksejkitajskij/Desktop/mangainua_parser/pdf ===


: 